In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors.kde import KernelDensity
from sklearn.cluster import DBSCAN
import itertools
import ast
from scipy import spatial
from scipy import special
import math

Read the data to a panda dataframe df and seperate labels

In [2]:
df = pd.read_csv('glass.data', sep=",", header=None)
label = df[10]
cols = [0,10]
df.drop(df.columns[cols],axis=1,inplace=True)
df.columns = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]

In [3]:
from sklearn import preprocessing

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)


In [3]:
df.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000
25%,1.516523,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000


In [14]:
data = df.as_matrix()

In [12]:
def calculate_expected_density(n, no_of_features, eps_param) :
    c = math.pow(math.pi , no_of_features/2) / special.gamma(no_of_features/2 + 1)
    v = 75.41
    exp = 2*n*math.pow(eps_param, no_of_features)*c/(math.pow(v, no_of_features)*(no_of_features + 2))
    return exp

In [5]:
def epanechnikov_kernel(data,data_point, tree , eps_param) :
    sum = 0
    N = tree.query_ball_point(data_point , eps_param)
    for j in N :
        x = np.linalg.norm(data_point-data[j])
        x = x/eps_param
        sum = sum + x*x    
    sum = len(N) - sum    
    return sum

In [6]:
def check_density(data,tree, data_point, eps_param, eta, F, expected_density, w) :
    val = epanechnikov_kernel(data,data_point , tree , eps_param)
    if val >= max(F*expected_density , eta*w) :
        return True
    return False

In [7]:
UNCLASSIFIED = False
NOISE = -1

def _expand_cluster(m, classifications, point_id, cluster_id, eps, min_points ,tree, eta, F,  expected_density, w):
    seeds = tree.query_ball_point(m[:,point_id] , eps)
    if len(seeds) < min_points or not check_density(m.transpose(),tree, m[:,point_id], eps, eta, F, expected_density, w):
        classifications[point_id] = NOISE
        return False
    else:
        classifications[point_id] = cluster_id
        for seed_id in seeds:
            classifications[seed_id] = cluster_id
            
        while len(seeds) > 0:
            current_point = seeds[0]
            results = tree.query_ball_point(m[:,current_point] , eps)
            if len(results) >= min_points:
                for i in range(0, len(results)):
                    result_point = results[i]
                    if classifications[result_point] == UNCLASSIFIED or \
                       classifications[result_point] == NOISE:
                        if classifications[result_point] == UNCLASSIFIED:
                            seeds.append(result_point)
                        classifications[result_point] = cluster_id
            seeds = seeds[1:]
        return True
        
def dbscan(m, eps, min_points, eta, F ):
    cluster_id = 1
    n_points = m.shape[1]
    classifications = [UNCLASSIFIED] * n_points
    
    tree = spatial.KDTree(m.transpose())
    expected_density = calculate_expected_density(m.shape[1], m.shape[0], eps)
    w = 2/(2  + m.shape[0])
    
    for point_id in range(0, n_points):
        point = m[:,point_id]
        if classifications[point_id] == UNCLASSIFIED:
            if _expand_cluster(m, classifications, point_id, cluster_id, eps, min_points ,tree, eta, F,  expected_density, w):
                cluster_id = cluster_id + 1
    return classifications


In [8]:
def dbscan_algo(data, eps_param, min_points,eta, F) :
    Clusters = []
    #db = DBSCAN(eps =eps_param, min_samples=min_points).fit(data)
    #labels = db.labels_
    
    labels = dbscan(data.transpose(), eps_param, min_points,eta, F)
    
    no_of_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    labels_present = list(set(labels))
    if -1 in labels_present :
        labels_present.remove(-1)
    #print("no_of_clusters : "+str(no_of_clusters))
    #print("labels_present : "+str(labels_present))
    c = {}
    for i in labels_present :
        c[i] = []
    for index, label in enumerate(labels) :
        if label != -1 :
            c[label].append(index)
    for i in labels_present :
        Clusters.append(c[i])
    return Clusters

In [9]:
def find_subsets(S,m) :
    return set(itertools.combinations(S , m))

In [10]:
def remove_redundancy(Clusters , r) :
    for feature_set_subspace, cluster_subspace in Clusters.items():
        #print("feature_set_subspace : "+str(feature_set_subspace))
        if len(cluster_subspace) == 0:
            continue
            
        feature_set_subspace = ast.literal_eval(feature_set_subspace)
        no_of_attr = len(feature_set_subspace)
        
            
        for length in range(1 , no_of_attr) :
            subpace_subsets = find_subsets(feature_set_subspace , length)
            for subset in subpace_subsets :
                
                #subset = set(subset)
                #print("subset : "+str(subset))
                
                if str(subset) not in Clusters.keys() :
                    #print("Entered2")
                    continue
                
                #print("Entered3")
                cluster_subset = Clusters[str(subset)]
                #print("Cluster_subset len : "+str(cluster_subset))
                if len(cluster_subset) == 0 :
                    continue
                
                remove_list = []
                for i in range(len(cluster_subset))  :
                    #print("len(cluster_subset[i]) : "+str(len(cluster_subset[i])))
                    for j in range(len(cluster_subspace)):
                        #print("len(cluster_subspace[j]) : "+str(len(cluster_subspace[j])))
                        if set(cluster_subspace[j]).issubset(cluster_subset[i]) :
                            #print("Entered1")
                            if len(cluster_subspace[j]) >= r * len(cluster_subset[i]) :
                                remove_list.append(i)
                                break
                
                for i in list(reversed(remove_list)) :
                    cluster_subset.pop(i)
                Clusters[str(subset)] = cluster_subset
                #print("New cluster_subset2 : "+str(Clusters[str(subset)]))
            
    return Clusters
                        
                    

In [11]:
def dusc_algo(data, eps_param, min_points, eta, r, F) :
    Clusters = {}
    total_no_of_features = data.shape[1]
    total_feature_set = range(total_no_of_features)
    
    #print( "total_no_of_features : "+str(total_no_of_features) )
    #print("total_feature_set : "+str(total_feature_set))
    
    for no_of_features in reversed(range(1 , total_no_of_features+1)) :
        for feature_set in find_subsets(total_feature_set, no_of_features) :
            #print("Selected feature set : "+str(feature_set))
            data_subspace = data[:, feature_set]
            #print("Complete data in this subspace : "+str(data_subspace))
            #data_subspace = find_dense_points(data_subspace, eps_param, eta, F)
            #print("Dense data in this subspace : "+str(data_subspace))
            
            if data_subspace.size == 0 :
                continue
            
            #print("Finding clusters in this subspace using DBSCAN")
            clusters_subspace = dbscan_algo(data_subspace,  eps_param, min_points,eta, F)
            #print("Clusters found in this subspace")
            #print(clusters_subspace)
            if len(clusters_subspace) > 0:
                Clusters[str(feature_set)] = clusters_subspace
    
    Clusters_list_all_supspaces = list(Clusters.values())
    Clusters_list_all_supspaces = [x for x in Clusters_list_all_supspaces if x != []]
    Clusters_list_redundant = []
    for l in Clusters_list_all_supspaces :
        Clusters_list_redundant = Clusters_list_redundant + l
    
    print("No of clusters before removing redundancy : "+str(len(Clusters_list_redundant)))
    Clusters = remove_redundancy(Clusters , r)     
    # now form list  of clusters from dictionary
    #print("Clusters after removing redundancy : ")
    #print(Clusters)
    if bool(Clusters) == False :
        print("No clusters found!!") 
        return {} , []
    
    Clusters_list_all_supspaces = list(Clusters.values())
    Clusters_list_all_supspaces = [x for x in Clusters_list_all_supspaces if x != []]
    Clusters_list = []
    for l in Clusters_list_all_supspaces :
        Clusters_list = Clusters_list + l
    print("No of clusters after removing redundancy : "+str(len(Clusters_list)))
    
    return Clusters , Clusters_list         


Parameter Initialization

In [14]:
eps = 0.2
min_points = 15
eta = 2
r = 0.1
F = 55

In [15]:
Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)

No of clusters before removing redundancy : 466
No of clusters after removing redundancy : 7


In [24]:
included = set()
for l in Clusters_list :
    for att in l :
        included.add(att)

coverage = len(included)*100.0/(data.shape[0])
print coverage

162
75.7009345794


In [28]:
eps = 0.2
min_points = 15
eta = 2
r = 0
F = 55
Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)

No of clusters before removing redundancy : 466
No of clusters after removing redundancy : 7


In [29]:
included = set()
for l in Clusters_list :
    for att in l :
        included.add(att)

coverage = len(included)*100.0/(data.shape[0])
print coverage

75.7009345794


In [26]:
eps = 0.15
min_points = 50
eta = 2
r = 0.1
F = 55
Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)

included = set()
for l in Clusters_list :
    print len(l)
    print l
    for att in l :
        included.add(att)

coverage = len(included)*100.0/(data.shape[0])
print coverage

No of clusters before removing redundancy : 55
No of clusters after removing redundancy : 6
85
[1, 3, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 40, 41, 42, 45, 46, 49, 51, 57, 58, 59, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 89, 91, 94, 95, 96, 117, 119, 120, 121, 122, 123, 124, 125, 126, 137, 138, 139, 140, 141, 143, 147, 148, 149, 150, 153, 154, 155, 156, 158, 159, 160, 187]
50
[1, 2, 5, 9, 11, 19, 20, 22, 23, 24, 25, 29, 32, 34, 35, 37, 42, 44, 52, 53, 54, 56, 57, 58, 59, 73, 77, 78, 79, 82, 83, 88, 99, 101, 121, 122, 124, 126, 134, 137, 141, 142, 144, 146, 149, 152, 155, 165, 177, 182]
64
[0, 17, 18, 21, 38, 39, 43, 47, 48, 50, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 92, 103, 104, 108, 109, 110, 111, 112, 131, 146, 151, 152, 157, 176, 177, 178, 179, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213]
56
[1, 3, 4, 6, 8, 9, 11,

In [25]:
eps = 0.15
min_points = 50
eta = 2
r = 0
F = 55
Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)

included = set()
for l in Clusters_list :
    print len(l)
    print l
    for att in l :
        included.add(att)

coverage = len(included)*100.0/(data.shape[0])
print coverage

No of clusters before removing redundancy : 55
No of clusters after removing redundancy : 6
85
[1, 3, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 40, 41, 42, 45, 46, 49, 51, 57, 58, 59, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 89, 91, 94, 95, 96, 117, 119, 120, 121, 122, 123, 124, 125, 126, 137, 138, 139, 140, 141, 143, 147, 148, 149, 150, 153, 154, 155, 156, 158, 159, 160, 187]
50
[1, 2, 5, 9, 11, 19, 20, 22, 23, 24, 25, 29, 32, 34, 35, 37, 42, 44, 52, 53, 54, 56, 57, 58, 59, 73, 77, 78, 79, 82, 83, 88, 99, 101, 121, 122, 124, 126, 134, 137, 141, 142, 144, 146, 149, 152, 155, 165, 177, 182]
64
[0, 17, 18, 21, 38, 39, 43, 47, 48, 50, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 92, 103, 104, 108, 109, 110, 111, 112, 131, 146, 151, 152, 157, 176, 177, 178, 179, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213]
56
[1, 3, 4, 6, 8, 9, 11,

In [ ]:
eps = 0.2
min_points = 14
eta = 2
r = 0.1
F = 55
Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)

included = set()
for l in Clusters_list :
    print len(l)
    print l
    for att in l :
        included.add(att)

coverage = len(included)*100.0/(data.shape[0])
print coverage

In [34]:
import math

In [35]:
def Quality(input_list) :
    entropy_list = []
    total_entropy = 0
    total_obj = 0
    for l in input_list :
        temp = list(set(l))
        entropy = 0
        for x in temp :
            t = l.count(x)/len(l)
            entropy = entropy + (t*math.log(t))
        normalized_entropy = (1+entropy)/math.log(len(temp))
        entropy_list.append(normalized_entropy)
        total_entropy =total_entropy + (-normalized_entropy*len(l))
        total_obj = total_obj + len(l)
    print(entropy_list)
    print(total_entropy)
    print(total_obj)
    print(total_entropy/total_obj)

In [38]:
Quality(Clusters_list)

ValueError: math domain error

In [39]:
min_pointss = [13,14,15]
for min_points in min_pointss :
    print(eps," ", min_points)
    Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)
    
epss = [0.15,0.15,0.15,0.19,0.25]
min_pointss = [13,16,15,11,10]

for i in range(5) : 
    eps = epss[i]
    min_points = min_pointss[i]
    print(eps," ", min_points)
    Clusters , Clusters_list = dusc_algo(data, eps, min_points, eta, r, F)
    
    included = set()
    for l in Clusters_list :
        for att in l :
            included.add(att)

    coverage = len(included)*100.0/(data.shape[0])
    print coverage
    

(0.2, ' ', 13)


KeyboardInterrupt: 

In [18]:
Clusters = {}
Clusters[str([1,2,3,4])] = [[2,3,4,5,6,7,8,9,10,11] , [1,10,3]]
Clusters[str([1,2])] = [[2,3,4,5,6,7,8,9,10,11,12,13,14] , [20,21,23]]

In [141]:
Clusters_list_all_supspaces = list(Clusters.values())
Clusters_list_all_supspaces = [x for x in Clusters_list_all_supspaces if x != []]
Clusters_list_redundant = []
for l in Clusters_list_all_supspaces :
    Clusters_list_redundant = Clusters_list_redundant + l
    

In [201]:
Clusters2 = remove_redundancy(Clusters , 0.1)

In [ ]:
for key , value in Clusters.items() :
    print(key)
    print(value)

In [30]:
for l in Clusters_list :
    print("len of l : "+str(len(l)))
    print(l)

len of l : 135
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 177, 187]
len of l : 15
[109, 182, 183, 191, 192, 195, 196, 197, 198, 199, 200, 202, 204, 208, 209]
len of l : 15
[195, 196, 197, 198, 199, 200, 202, 204, 205, 206, 208, 209, 210, 212, 213]
len of l : 15
[195, 196, 197, 198, 199, 200, 202, 204, 205, 206, 208, 209, 210, 211, 212]
len of l : 19
[191, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 208, 210, 211, 212, 213]
len of l : 17
[110, 111, 16